In [56]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [57]:
# Create the optimization model
model = gb.Model("Question 1: Vaccines")

In [58]:
Billy_Bishop_Toronto_City_Airport_costs = [0.05, 0.05, 0.05, 0.05, 0.05, 0.06, 0.06, 0.06, 0.06, 0.06, 0.07, 0.07, 0.07, 0.07, 0.07, 0.08, 0.08, 0.08, 0.08, 0.08, 0.09, 0.09, 0.09, 0.09, 0.09, 0.1, 0.1, 0.1, 0.1]
Toronto_Pearson_Airport_costs = [0.08, 0.08, 0.08, 0.08, 0.08, 0.05, 0.05, 0.05, 0.05, 0.05, 0.09, 0.09, 0.09, 0.09, 0.09, 0.1, 0.1, 0.1, 0.1, 0.1, 0.07, 0.07, 0.07, 0.07, 0.07, 0.06, 0.06, 0.06, 0.06]

In [59]:
# Create the a single class of decision variables where
# From = {Billy} and To = {29 sites}.
x = model.addVars(29, lb=0, vtype=GRB.CONTINUOUS, name="Billy Shipping")
# From = {Pearson} and To = {29 sites}.
y = model.addVars(29, lb=0, vtype=GRB.CONTINUOUS, name="Pearson Shipping")

In [60]:
Billy_objective = gb.quicksum(Billy_Bishop_Toronto_City_Airport_costs[j] * x[j] for j in range(29))
Pearson_objective = gb.quicksum(Toronto_Pearson_Airport_costs[j] * y[j] for j in range(29))
model.setObjective(Billy_objective + Pearson_objective, GRB.MINIMIZE)

In [61]:
absolute_value_1 = model.addConstr(gb.quicksum(x[j] for j in range(5)) - gb.quicksum(y[j] for j in range(5)) <= 4800, name="4800 constraint")
absolute_value_2 = model.addConstr(gb.quicksum(y[j] for j in range(5)) - gb.quicksum(x[j] for j in range(5)) <= 4800, name="4800 constraint negative")

In [62]:
ratio_1 = model.addConstr(gb.quicksum(y[j] for j in range(20,25)) <= 8 * (gb.quicksum(x[j] for j in range(10,15))), name="Second constraint")

In [63]:
ratio_2 = model.addConstr(gb.quicksum(x[j] for j in range(25,29)) >= 0.8 * (gb.quicksum(y[j] for j in range(15,20))), name="Third constraint")

In [64]:
model.addConstr(gb.quicksum(x[j] for j in range(29)) == 100000, "Billy Supply Constraint")
model.addConstr(gb.quicksum(y[j] for j in range(29)) == 250000, "Pearson Supply Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [65]:
#Hosipital Constraint
#model.addConstr((gb.quicksum(x[j] + y[j] for j in range(7))) == 4 * (gb.quicksum(x[j] + y[j] for j in range(7,29))), "Hosipital Constraint")

In [66]:
#Hosipital Constraint
# 50000 = 7x + 22x/4
# x = 4000
# 4000 per day, 7 days = 28000 per week
for j in range(7):
    model.addConstr(x[j] + y[j] <= 28000, f"Hospital Constraint_{j}")

# 50000 = 4*7x + 22x
# x = 1000
# 1000 per day, 7 days = 7000 per week
for j in range(7,29):
    model.addConstr(x[j] + y[j] <= 7000, f"Non-Hospital Constraint_{j}")

In [67]:
# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 35 rows, 58 columns and 155 nonzeros
Model fingerprint: 0xc46d81a7
Coefficient statistics:
  Matrix range     [8e-01, 8e+00]
  Objective range  [5e-02, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 3e+05]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 34 rows, 59 columns, 146 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7500000e+04   8.539300e+04   0.000000e+00      0s
      56    2.4828000e+04   0.000000e+00   0.000000e+00      0s

Solved in 56 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.482800000e+04


In [68]:
# Print the decision variables
print(model.printAttr('X'))


    Variable            X 
-------------------------
Billy Shipping[0]        28000 
Billy Shipping[2]        28000 
Billy Shipping[3]        11600 
Billy Shipping[10]         4375 
Billy Shipping[19]          125 
Billy Shipping[25]         7000 
Billy Shipping[26]         7000 
Billy Shipping[27]         7000 
Billy Shipping[28]         6900 
Pearson Shipping[1]        28000 
Pearson Shipping[3]        16400 
Pearson Shipping[4]        28000 
Pearson Shipping[5]        28000 
Pearson Shipping[6]        28000 
Pearson Shipping[7]         7000 
Pearson Shipping[8]         7000 
Pearson Shipping[9]         7000 
Pearson Shipping[10]         2625 
Pearson Shipping[11]         7000 
Pearson Shipping[12]         7000 
Pearson Shipping[13]         7000 
Pearson Shipping[14]         7000 
Pearson Shipping[15]         7000 
Pearson Shipping[16]         7000 
Pearson Shipping[17]         7000 
Pearson Shipping[18]         7000 
Pearson Shipping[19]         6875 
Pearson Shipping[20]         7

In [69]:
# Get the optimal values of decision variables
billy = model.getAttr('x', x)
billy

{0: 28000.0,
 1: 0.0,
 2: 28000.0,
 3: 11600.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 4375.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 124.99999999999818,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 7000.0,
 26: 7000.0,
 27: 7000.0,
 28: 6900.000000000002}

In [70]:
# Get the optimal values of decision variables
pearson = model.getAttr('x', y)
pearson

{0: 0.0,
 1: 28000.0,
 2: 0.0,
 3: 16400.0,
 4: 28000.0,
 5: 28000.0,
 6: 28000.0,
 7: 7000.0,
 8: 7000.0,
 9: 7000.0,
 10: 2625.0,
 11: 7000.0,
 12: 7000.0,
 13: 7000.0,
 14: 7000.0,
 15: 7000.0,
 16: 7000.0,
 17: 7000.0,
 18: 7000.0,
 19: 6875.000000000002,
 20: 7000.0,
 21: 7000.0,
 22: 7000.0,
 23: 7000.0,
 24: 7000.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 99.99999999999818}

In [71]:
# Value of the objective function
print("Total Transportation cost: ", round(model.objVal, 2))

Total Transportation cost:  24828.0


a) 28000 as noted by the constraint.

b) The max is 7000 as seen in the constraint, and it does not appear any site is lower, so 7000.

c) 58 decision variables, 29 for billy and 29 for pearson.

d) Absolute Value constraint (it needed to be written twice).

e) Code: model.addConstr(gb.quicksum(y[j] for j in range(29)) == 250000, "Pearson Supply Constraint")
Math: i = 29 sigma(y_i) = 250000

f) Code: model.addConstr(gb.quicksum(x[j] for j in range(25,29)) >= 0.8 * (gb.quicksum(y[j] for j in range(15,20))), name="Third constraint")
Math: i = 25-28 sigma(x_i) >= 0.8*(i=15-19 sigma(y_i))

g) The optimal function value is $24828

h) I'm assuming he meant site 4, 16400

In [72]:
print("")
print(f"Sensitivity Information for Land Capacity Constraint {ratio_1.pi:.2f}:")
print("(LHS, RHS, Slack): ", (model.getRow(ratio_1).getValue(), ratio_1.RHS, ratio_1.slack))
print("Shadow Price: ", ratio_1.pi)
print("Range of Feasibility: ", (ratio_1.SARHSUp, ratio_1.SARHSLow))


Sensitivity Information for Land Capacity Constraint -0.03:
(LHS, RHS, Slack):  (0.0, 0.0, 0.0)
Shadow Price:  -0.030000000000000013
Range of Feasibility:  (11000.000000000002, -199.9999999999963)


In [73]:
print("")
print(f"Sensitivity Information for Land Capacity Constraint {ratio_2.pi:.2f}:")
print("(LHS, RHS, Slack): ", (model.getRow(ratio_2).getValue(), ratio_2.RHS, ratio_2.slack))
print("Shadow Price: ", ratio_2.pi)
print("Range of Feasibility: ", (ratio_2.SARHSUp, ratio_2.SARHSLow))


Sensitivity Information for Land Capacity Constraint 0.30:
(LHS, RHS, Slack):  (-1.8189894035458565e-12, 0.0, 0.0)
Shadow Price:  0.3000000000000001
Range of Feasibility:  (19.999999999999634, -1375.0000000000002)


i) Probably an easier way to tell, but they are binding since the shadow prices are not 0

In [74]:
print("Shadow Price: ", absolute_value_1.pi)
print("Shadow Price: ", absolute_value_2.pi)

Shadow Price:  0.0
Shadow Price:  -0.11500000000000003


j) The absolute value and ratio constraints are binding, affecting the problem. While supply constraints are necessary for the problem, the demand constraints for each site is what makes the problem work the way it does. Removing them would probably result in lower cost.